<a href="https://colab.research.google.com/github/sepengsu/robotics_project/blob/main/foundations_of_robotics_practice_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Requirements
- At the first run, "The `rerun` module is missing. Installing now via pip..." message appears, and rerun library which is needed for visualization will be installed.
- After installing library, run block again.
- If "successfully imported rerun.io" message is printed, required library is successfully installed and imported.

In [ ]:
try:
  import rerun as rr
  print("successfully imported rerun.io")
except ImportError:
  print("The `rerun` module is missing. Installing now via pip...")
  # These versions of numpy and pandas-gbq are compatible with the rest of the colab environment
  !pip install numpy>=1.23 pandas-gbq==0.19 rerun-sdk==0.14.1
  print('Installation completed. The runtime needs to be restarted. Stopping now.')
  import os
  os.kill(os.getpid(), 9)

successfully imported rerun.io


In [ ]:
import numpy as np

def load_checkpoints():
  checkpoints = [[2,1],
                 [3,2],
                 [2,4],
                 [1,5],
                 [3,4.5],
                 [4,6],
                 [5,4],
                 [8,3],
                 [7,2],
                 [12,1],
                 [11,3],
                 [13,4],
                 [11,5],
                 [11,7],
                 [8,6],
                 [8,9],
                 [4,10],
                 [5,13],
                 [14,14]]
  return np.array(checkpoints)

def evaluate(checkpoints, robot_traj, delta_t):
  robot_traj = np.array(robot_traj)
  matched_idxs = []
  ATE_errors = []
  last_matched_idx = 0
  len_traj = len(robot_traj)
  for checkpoint in checkpoints:
    errors = np.sum((np.repeat(checkpoint.reshape(1,2), len_traj-last_matched_idx, axis=0) - robot_traj[last_matched_idx:,:2])**2, axis=1)
    matched_idx = np.argmin(errors,
                            axis=0)
    matched_idxs.append(matched_idx + last_matched_idx)
    ATE_errors.append(errors[matched_idx])
    last_matched_idx = matched_idx

  # ATE RMSE
  ATE_RMSE = np.sqrt(np.mean(ATE_errors))
  print(f"ATE RMSE: {ATE_RMSE:.4f}")

  # time
  time = delta_t * (len_traj-1)
  print(f"Time: {time}")

  return np.array(matched_idxs)

class Robot:
  def __init__(self):
    self.R = 0.03 # wheel radius
    self.D = 0.20 # distance between two wheels
    self.pulses_per_turn = 72
    self.traj = [[0., 0., 0.]] # x, y, phi

  def get_wheels_speed(self, encoderValues, oldEncoderValues, delta_t):
    # Calculate the change in angular position of the wheels:
    ang_diff_l = 2*np.pi*(encoderValues[0] - oldEncoderValues[0])/self.pulses_per_turn
    ang_diff_r = 2*np.pi*(encoderValues[1] - oldEncoderValues[1])/self.pulses_per_turn

    # Calculate the angular speeds:
    wl = ang_diff_l/delta_t
    wr = ang_diff_r/delta_t

    return wl, wr

  def get_robot_speeds(self, wl, wr):
    u = self.R/2.0 * (wr + wl)
    w = self.R/self.D * (wr - wl)
    return u, w

  def get_robot_pose(self, u, w, delta_t):
    # get previous pose
    x_old, y_old, phi_old = self.traj[-1]

    delta_phi = w * delta_t
    phi = phi_old + delta_phi

    if phi >= np.pi:
        phi = phi - 2*np.pi
    elif phi < -np.pi:
        phi = phi + 2*np.pi

    delta_x = u * np.cos(phi) * delta_t
    delta_y = u * np.sin(phi) * delta_t
    x = x_old + delta_x
    y = y_old + delta_y

    return x, y, phi

  def update_pose(self, encoderValues, oldEncoderValues, delta_t):
    # odometry -> pose update
    wl, wr = self.get_wheels_speed(encoderValues, oldEncoderValues, delta_t)
    u, w = self.get_robot_speeds(wl, wr)
    current_pose = self.get_robot_pose(u, w, delta_t)
    # add calculated current pose to pose history
    self.traj.append(current_pose)

  def wheel_speed_commands(self, u_d, w_d):
    wr_d = float((2*u_d + self.D*w_d)/(2*self.R))
    wl_d = float((2*u_d - self.D*w_d)/(2*self.R))

    return wl_d, wr_d

  def move_robot(self, wl, wr, delta_t):
    # odometry -> pose update

    u, w = self.get_robot_speeds(wl, wr)
    current_pose = self.get_robot_pose(u, w, delta_t)
    self.traj.append(current_pose)

  def get_pose_error(self, xd, yd):

    # Position error:
    x_err = xd - self.traj[-1][0]
    y_err = yd - self.traj[-1][1]
    dist_err = np.sqrt(x_err**2 + y_err**2)

    # Orientation error
    phi_d = np.arctan2(y_err,x_err)
    phi_err = phi_d - self.traj[-1][2]

    # Limits the error to (-pi, pi):
    phi_err_correct = np.arctan2(np.sin(phi_err),np.cos(phi_err))

    return dist_err, phi_err_correct

  ##### Implement your own controller!! #####
  def controller(self, phi_err_correct, delta_t):
    kp_phi = 0.8
    P_phi = kp_phi*phi_err_correct
    output_phi = P_phi

    return output_phi
  ##### Implement your own controller!! #####

def manage_checkpoint(checkpoints, curr_checkpoint_idx, curr_pose):
  curr_checkpoint = checkpoints[curr_checkpoint_idx]
  dist_err = np.sqrt(((curr_checkpoint-curr_pose)**2).sum())
  if dist_err < 0.2:
    target_checkpoint_idx = curr_checkpoint_idx+1
  else:
    target_checkpoint_idx = curr_checkpoint_idx
  return target_checkpoint_idx

robot1 = Robot()
rr.init("cube")
rec = rr.memory_recording()

# Load_checkpoints
checkpoints = load_checkpoints()
target_checkpoint_idx = 0

delta_t = 0.1
step = 0
xd, yd = checkpoints[target_checkpoint_idx]
dist_err, phi_err_correct = robot1.get_pose_error(xd, yd)

while True:
  ############ Visualize ###########
  rr.set_time_sequence("step", step)
  rr.log("none", rr.Points2D([-0.5,-0.5], radii=1e-5))
  rr.log("x", rr.Arrows2D(origins=[0,0], vectors=[15.0, 0], colors=[255,0,0]))
  rr.log("y", rr.Arrows2D(origins=[0,0], vectors=[0, 15.0], colors=[0,255,0]))

  ## robot pose
  rr.log("body", rr.Points2D([robot1.traj[-1][0], robot1.traj[-1][1]],
                             radii=0.3,
                             colors=[241,128,45]))
  rr.log("heading", rr.Arrows2D(origins=robot1.traj[-1][:2],
                          vectors=[np.cos(robot1.traj[-1][2]), np.sin(robot1.traj[-1][2])],
                          radii=0.1,
                          colors=[241,128,45]))
  ## desired position
  rr.log("desired points", rr.Points2D(checkpoints,
                                       radii=0.2,
                                       colors=[219,226,239]))
  ## pose to desired position
  rr.log("error", rr.Arrows2D(origins=robot1.traj[-1][:2],
                        vectors=[dist_err*np.cos(phi_err_correct+robot1.traj[-1][2]), dist_err*np.sin(phi_err_correct+robot1.traj[-1][2])],
                        radii=0.03))
  ############ Visualize ###########

  w_d = robot1.controller(phi_err_correct, delta_t)
  wl_d, wr_d = robot1.wheel_speed_commands(0.3, w_d)

  robot1.move_robot(wl_d, wr_d, delta_t)

  target_checkpoint_idx = manage_checkpoint(checkpoints, target_checkpoint_idx, robot1.traj[-1][:2])
  if target_checkpoint_idx >= len(checkpoints):
    break
  xd, yd = checkpoints[target_checkpoint_idx]
  dist_err, phi_err_correct = robot1.get_pose_error(xd, yd)

  step += 1

# evaluate
matched_idxs = evaluate(checkpoints, robot1.traj, delta_t)

# Visualize robot trajectory / matched poses
robot_traj = np.array(robot1.traj)
rr.log("robot traj", rr.Points2D(robot_traj[:,:2],
                                 radii=0.05,
                                 colors=[84,154,171]))
rr.log("matched traj", rr.Points2D(robot_traj[matched_idxs,:2],
                                   radii=0.15,
                                   colors=[241,128,45]))
rec


ATE RMSE: 0.0858
Time: 183.10000000000002
